In [1]:
import pandas as pd
import numpy as np
import os 
import cv2
from skimage.io import ImageCollection
from skimage.io import imread,imshow, imread_collection
from utils_detector import DataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

/home/wvillegas/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
dataset = pd.read_csv(os.path.join('/home','wvillegas','dataset-mask', 'full_masks.csv'))

In [3]:
masks_path = os.path.join('/home','wvillegas','dataset-mask','masks')
img_path = os.path.join('/home','wvillegas','dataset-mask','images')

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(dataset['orig'],dataset['mask'],
                                                    test_size = 0.2, random_state=1)

In [5]:
partition = {'train':list(X_train),
             'test': list(X_test)}

In [6]:
img_list = list(X_train) + list(X_test)
mask_list = list(Y_train) + list(Y_test)

In [7]:
labels = dict(zip(img_list, mask_list))

In [8]:
batch_size = 1

In [9]:
train_generator = DataGenerator(batch_size=batch_size,img_path=img_path, labels=labels,
                                list_IDs=partition['train'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)
valid_generator = DataGenerator(batch_size=batch_size,img_path=img_path, labels=labels,
                                list_IDs=partition['test'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)

In [10]:
from keras.applications import VGG16
from keras.layers import Conv2DTranspose,Conv2D
from keras import Model

In [11]:
vgg = VGG16(include_top=False,weights='imagenet')

In [12]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [13]:
# t1 = Conv2DTranspose(1,activation='relu',kernel_size=4)(mobilenet.output)
# t2 = Conv2DTranspose(1,activation='relu',kernel_size=8)(t1)
# t3 = Conv2DTranspose(1,activation='relu',kernel_size=16)(t2)

# fin = Conv2D(1,1,activation='sigmoid')(t3)

In [14]:
# test arquitectura paper de FCN 
# deconv1 = Conv2DTranspose(filters=128,kernel_size=3,strides=2,activation='relu')(mobilenet.output)
# deconv2 = Conv2DTranspose(filters=64,kernel_size=3,strides=2,activation='relu')(deconv1)
deconv3 = Conv2DTranspose(filters=32,kernel_size=3,strides=2,activation='relu')(vgg.output)
deconv4 = Conv2DTranspose(filters=16,kernel_size=3,strides=2,activation='relu')(deconv3)
deconv5 = Conv2DTranspose(filters=8,kernel_size=4,strides=3,activation='relu')(deconv4)
final = Conv2D(1,1,activation='sigmoid')(deconv5)

In [15]:
fcn = Model(inputs=vgg.input,outputs=final)

In [16]:
fcn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [17]:
# plt.imshow(pred[0,:,:,0], cmap="gray")

In [18]:
from keras.optimizers import SGD, RMSprop

In [19]:
rms = SGD(lr=0.001, momentum=0.9)
fcn.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

In [20]:
history = fcn.fit_generator(generator=train_generator,validation_data=valid_generator,epochs=1)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,3,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/SGD/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](SGD/momentum/read, training/SGD/Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics/acc/Mean_1/_279 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1313_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
pred = fcn.predict(X[0:1,:,:,:])

In [ ]:
pred.shape

In [ ]:
plt.imshow(pred[0,:,:,0], cmap="gray")

In [ ]:
imshow(Y[0][:,:,0])